In [ ]:
from sa_xla import SplashAttentionWrapper,SPLASH_ATTENTION_AVAILABLE,SplashAttentionConfig
from transformers.models.qwen2.modeling_qwen2 import apply_rotary_pos_emb ## some may have thier own 
model = AutoModelForCausalLM.from_pretrained(MODEL,torch_dtype=torch.bfloat16,trust_remote_code=True)

for layer in model.model.layers:
    xs.apply_backward_optimization_barrier(layer)
 
logits_soft_cap: Optional[float] = None

## mesh is ((fsdp,4),(data,1)) 
## Splash need doesn't support 2D mesh yet 
### Similar to Flash Attetion ; GQA and AdamW Simultaneously not supported 
## Requires torch-xla,torch-xla['pallas'] and torchax 
   
with torch.no_grad():
    if SPLASH_ATTENTION_AVAILABLE:

        config_default=SplashAttentionConfig(
            mesh=str(xs.get_global_mesh()),
            qkv_partition_spec=(("fsdp", "data"),None ,None, None),
            segment_ids_partition_spec=(("fsdp", "data"),None))
                
        config_2048 = SplashAttentionConfig(
            mesh=str(xs.get_global_mesh()),
            qkv_partition_spec=(("fsdp", "data"),None ,None, None),
            segment_ids_partition_spec=(("fsdp", "data"),None),
            sa_block_q=512,
            sa_block_kv=512,
            sa_block_kv_compute=512,
            sa_block_q_dkv=512,
            sa_block_kv_dkv=512,
            sa_block_kv_dkv_compute=512,
            sa_block_q_dq=512,
            sa_block_kv_dq=512,
        )
        
        config_1024 = SplashAttentionConfig(  ## use this for large context size
            mesh=str(xs.get_global_mesh()),
            qkv_partition_spec=(("fsdp", "data"),None ,None, None),
            segment_ids_partition_spec=(("fsdp", "data"),None),
            sa_block_q=256,
            sa_block_kv=256,
            sa_block_kv_compute=256,
            sa_block_q_dkv=256,
            sa_block_kv_dkv=256,
            sa_block_kv_dkv_compute=256,
            sa_block_q_dq=256,
            sa_block_kv_dq=256,
        )
        

        old_atten_config=[]
        
        for layer in model.model.layers:
            original_attention = layer.self_attn

            layer.self_attn = SplashAttentionWrapper(original_attention, config=config_2048,
                                logits_soft_cap=logits_soft_cap,rotatry_func=apply_rotary_pos_emb)

            old_atten_config.append(original_attention)
        
        print("Applied Splash Attention for TPU ; Saved Old Attention config for replacement after training")


In [ ]:
## train....

In [ ]:
# Unwrap the XLA Flash Attention
model = model.cpu()
if SPLASH_ATTENTION_AVAILABLE :
    try:
        for index,attn in enumerate(old_atten_config):
            model.model.layers[index].self_attn=attn
        print("Replaced SplashAttentionWrapper With Original Attention Implementation")
    except:
        print("Unable to Replace SplashAttentionWrapper With Original Attention Implementation")
    model = model.cpu()

